# Greedy MSU selection

Choose MSU locations based on which improves overall utility most.

In [1]:
import numpy as np
import pandas as pd

from classes.geography_processing import Geoprocessing
from classes.model import Model
from classes.scenario import Scenario

# Turn off all warnings
import warnings
warnings.filterwarnings("ignore")


In [2]:
# Proportion of treated patients that are LVO
prop_lvo = 0.40

# Maximum number of hospitals to select
max_hospitals = 100

In [3]:
# Set up scenario
scenario = Scenario({
    'name': 1,
})

# Set up geoprocessing module and load initial data (and  set Use_MSU column to zero
geo = Geoprocessing(limit_to_england = True)
geo.load_data()
geo.hospitals['Use_MSU'] = 0

# Travel processing for constant locations
geo.find_nearest_ivt_unit()
geo.find_nearest_mt_unit()
geo.find_nearest_transfer_mt_unit()

# Set up list of selected and unselected MSUs
selected_MSUs = []
used_hospitals_mask = geo.hospitals['Use_IVT'] == 1
unelected_MSUs = list(geo.hospitals[used_hospitals_mask].index)

## Benefit from each hospital as a MSU base


In [4]:
hospitals = geo.hospitals.copy(deep=True)

# Loop through all MSUs
possible_msu_locations = list(geo.hospitals[used_hospitals_mask].index)
advantages = []

for msu in possible_msu_locations:
    data_copy = hospitals.copy(deep=True)
    data_copy.loc[msu, 'Use_MSU'] = 1
    geo.hospitals = data_copy.copy()
    geo.find_nearest_msu_unit()
    geo.collate_data()
    geo.save_processed_data()

    model = Model(
        scenario=scenario,
        geodata=pd.read_csv('processed_data/processed_data.csv')
    )

    model.run()

    # Get utilitites
    lsoa_msu_utility = (
        (model.full_results['lvo_msu_ivt_mt_utility'] * prop_lvo) + 
        (model.full_results['nlvo_msu_ivt_utility'] * (1 - prop_lvo)))

    lsoa_drip_ship_utility = (
        (model.full_results['lvo_drip_ship_ivt_mt_utility'] * prop_lvo) + 
        (model.full_results['nlvo_drip_ship_ivt_utility'] * (1 - prop_lvo)))

    # Get the maximum of MSU and drip and ship
    best_utility = np.maximum(lsoa_msu_utility, lsoa_drip_ship_utility)
    best_utility

    # Get the weighted average of lsoa_drip_ship_utility
    average_utility_drip_ship = np.sum(
        model.full_results['Admissions'] * lsoa_drip_ship_utility) / np.sum(model.full_results['Admissions'])

    # Get the average utility weighted by model.full_results['Admissions']
    average_utility = np.sum(model.full_results['Admissions'] * best_utility) / np.sum(model.full_results['Admissions'])

    # Get the advantage of MSU over drip and ship
    advantage = average_utility - average_utility_drip_ship

    advantages.append(advantage)


results = geo.hospitals.copy(deep=True)
results = results[used_hospitals_mask]
results['MSU advantage'] = advantages

results.to_csv('output/msu_advantages_individual.csv')

## Greedy selection of MSUs

In [ ]:
hospitals = geo.hospitals.copy(deep=True)
best_advantage = 0
advantages = []

# Loop through greedy selection process
while (len(unelected_MSUs) > 0) & (len(selected_MSUs) < max_hospitals):

    # Loop through all MSUs
    for msu in unelected_MSUs:
        data_copy = hospitals.copy(deep=True)
        data_copy.loc[msu, 'Use_MSU'] = 1
        geo.hospitals = data_copy.copy()
        geo.find_nearest_msu_unit()
        geo.collate_data()
        geo.save_processed_data()

        model = Model(
            scenario=scenario,
            geodata=pd.read_csv('processed_data/processed_data.csv')
        )

        model.run()
        
        # Get utilitites
        lsoa_msu_utility = (
            (model.full_results['lvo_msu_ivt_mt_utility'] * prop_lvo) + 
            (model.full_results['nlvo_msu_ivt_utility'] * (1 - prop_lvo)))

        lsoa_drip_ship_utility = (
            (model.full_results['lvo_drip_ship_ivt_mt_utility'] * prop_lvo) + 
            (model.full_results['nlvo_drip_ship_ivt_utility'] * (1 - prop_lvo)))

        # Get the maximum of MSU and drip and ship
        best_utility = np.maximum(lsoa_msu_utility, lsoa_drip_ship_utility)
        best_utility

        # Get the weighted average of lsoa_drip_ship_utility
        average_utility_drip_ship = np.sum(
            model.full_results['Admissions'] * lsoa_drip_ship_utility) / np.sum(model.full_results['Admissions'])

        # Get the average utility weighted by model.full_results['Admissions']
        average_utility = np.sum(model.full_results['Admissions'] * best_utility) / np.sum(model.full_results['Admissions'])

        # Get the advantage of MSU over drip and ship
        advantage = average_utility - average_utility_drip_ship
        
        if advantage > best_advantage:
            best_advantage = advantage
            best_MSU = msu

    # Add the best MSU to the selected list
    selected_MSUs.append(best_MSU)
    unelected_MSUs.remove(best_MSU)
    hospitals.loc[best_MSU, 'Use_MSU'] = 1
    advantages.append(best_advantage)

    # print the best MSU and utility
    print(f'{best_MSU}, {best_advantage:0.3f}')

In [ ]:
unelected_MSUs

In [ ]:
greedy_results = geo.hospitals.loc[selected_MSUs]
greedy_results['MSU advantage'] = advantages
greedy_results.to_csv('output/msu_advantages_greedy.csv', index=False)
greedy_results